# Sparse Subspace Clustering (Vidal IEEE Trans. on PAMI 2013) on Yale Dataset

#### References:


1.   Main Paper (Vidal IEEE Trans. on PAMI 2013): https://arxiv.org/pdf/1203.1005.pdf
2.   Supplementary Paper (Vidal CVPR 2009): http://cis.jhu.edu/~ehsan/Downloads/SSC-CVPR09-Ehsan.pdf
3.   Spectral Clustering: http://people.csail.mit.edu/dsontag/courses/ml14/notes/Luxburg07_tutorial_spectral_clustering.pdf
4.   Spectral Clustering Code: https://juanitorduz.github.io/spectral_clustering/
5.   Yale Dataset: http://vision.ucsd.edu/content/yale-face-database

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Our objective is given a set of points drawn from a `union of subspaces`, we need to find the number of subspaces, their dimensions, a basis for each subspace, and the segmentation of the data.

In [2]:
import numpy as np
import cvxpy as cp
from cvxpy.atoms.elementwise.power import power

##### Set Parameters of the data space

In [8]:
N = 165 # Total no of points
D = 1024  # Dimension of space
K = 15 # Number of clusters

In [9]:
yale_data = np.load('/content/drive/My Drive/yale_X.npy')
input_data = yale_data.astype('float32') / 255.
input_data = input_data.T
orig_label = np.load('/content/drive/My Drive/yale_Y.npy')

The matrix *input_data* contains the data points arranged sequentially forming a matrix of shape $(1024, 165), ie. (D,N)$. The *input_data* matrix is of the form $[y[0],y[1], \ldots, y[N-1]], $where each $y_i$ denotes the vector of a data point of dimension $D$.

Also, *input_data* can be assumed to be of the form $[Y_0, Y_1, \ldots, Y_n]$, where each $Y_i$ denotes the set of $N_i$ data points belonging to subspace $i$. Also assume that the dimension of each subspace $i$ is $d_i$ and $A_i$ is its basis.

In [10]:
def find_sparse_sol(Y,i,N):
    if i is 0:
        Ybari = Y[:,1:N]
    if i is N-1:
        Ybari = Y[:,0:N-1]
    if i!=0 and i!=N-1:
        Ybari = np.concatenate((Y[:,0:i],Y[:,i+1:N]),axis=1)
    yi = Y[:,i].reshape(D,1)
    
    # this ci will contain the solution of the l1 optimisation problem:  
    # min (||yi - Ybari*ci||F)^2 + lambda*||ci||1   st. sum(ci) = 1
    
    ci = cp.Variable(shape=(N-1,1))
    constraint = [cp.sum(ci)==1]
    obj = cp.Minimize(power(cp.norm(yi-Ybari*ci,2),2) + 199101*cp.norm(ci,1)) #lambda = 199101
    prob = cp.Problem(obj, constraint)
    prob.solve()
    return ci.value

The above function `find_sparse_sol` finds the solution to the following optimisation problem $(\textrm{min} \; ||y_i - Y_{\hat{i}}c_i||_{F}^2 + \lambda||c_i||_1)$ subject to the constraint $\sum_{i=i}^{N}c_i=1$. We use this optimisation term to *deal with all the possible noise values in the data.*

In [11]:
C = np.concatenate((np.zeros((1,1)),find_sparse_sol(input_data,0,N)),axis=0)

for i in range(1,N):
    ci = find_sparse_sol(input_data,i,N)
    zero_element = np.zeros((1,1))
    cif = np.concatenate((ci[0:i,:],zero_element,ci[i:N,:]),axis=0)
    C = np.concatenate((C,cif),axis=1)
print(C.shape)

(165, 165)


We now include a zero vector of size $(1,1)$ at the $i$ th position of $c_i$ to form $\hat{c_i}$ which we represent as *cif* in the code.
Then we concatenate $[\hat{c_1}$, $\hat{c_2}$, ..., $\hat{c_N}]$ to form the matrix $C$.  

$C$ is the *Matrix of Coefficients* and is of the form $C = [\hat{c_1}, \hat{c_2}, ..., \hat{c_N}] \in \mathbb{R}^{NXN}$.

In [12]:
W = np.add(np.absolute(C), np.absolute(C.T))
print(W.shape)

(165, 165)


In [13]:
# Check sparsity by counting the number of zeros

cz = 0
for i in range(W.shape[0]):
    for j in range(W.shape[1]):
        if W[i,j] < 1e-5 and W[i,j] > -1e-5:
            cz += 1
print(cz)

25137


In the above code block, we make $C$ a symmetric matrix by the operation $W = |C| + |C^T|$. It is still a valid representation of the similarity since if $y_i$ can write itself as a linear combination of all the points in the same subspace including $y_j$, then $y_j$ can also be represented as a linear combination of all the other points belonging to the same subspace including $c_i$.
            In the above code block, we print the number of $0$'s in the matrix $W$, we denote any $|W_{ij}|$ less then $e^{-5}$ as $0$.

In [14]:
D = np.zeros((N,N))
sum_list=[]
for i in range(N):
    csum = np.sum(W[:,i],axis=0)
    sum_list.append(csum)

D = np.diag(sum_list)
print(D)

[[2.13075991 0.         0.         ... 0.         0.         0.        ]
 [0.         2.70513964 0.         ... 0.         0.         0.        ]
 [0.         0.         2.48878329 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.36819346 0.         0.        ]
 [0.         0.         0.         ... 0.         1.50284329 0.        ]
 [0.         0.         0.         ... 0.         0.         1.5949385 ]]


$D$ is the `degree matrix`. In this case, $D \in \mathbb{R}^{NxN}$ is a diagonal matrix with $D_{ii} = \sum_{j}W_{ij}$. 

In [15]:
L = np.subtract(D, W)
print(L.shape)

(165, 165)


This $L$ is the Laplacian matrix, which can be defined as $L = D - W$. Next, we calculate the `eigenvalues` and `eigenvectors` of the Laplacian matrix, which we will use for Spectral clustering of the data points. $L$ is a *positive, semi-definite matrix* this means all the eigenvalues of the matrix will be greater than equal to $0$.

### Perform Spectral Clustering with Laplacian Matrix L

In [16]:
from scipy import linalg

eigenvals, eigenvcts = linalg.eig(L)

eigenvals = np.real(eigenvals)
eigenvcts = np.real(eigenvcts)

eig = eigenvals.reshape((N,1))

Sort Eigen Values

In [17]:
eigenvals_sorted_indices = np.argsort(eigenvals)
eigenvals_sorted = eigenvals[eigenvals_sorted_indices]

In [18]:
indices = []
for i in range(0,K):
    ind = []
    print(eigenvals_sorted_indices[i])
    ind.append(eigenvals_sorted_indices[i])
    indices.append(np.asarray(ind))

0
29
36
37
39
38
45
46
47
48
52
57
53
58
59


In the above code, we find out the indices of the eigenvectors corresponding to the $K$ smallest eigenvectors.

In [19]:
indices = np.asarray(indices)

In [20]:
zero_eigenvals_index = np.array(indices)

Here, the indices are arranged according to their sorted order of values and the sorted eigen values are stored in *eigenvals_sorted*.

In [21]:
eigenvals[zero_eigenvals_index]

array([[1.25855366e-15],
       [4.33867365e-02],
       [9.06083468e-02],
       [1.22842027e-01],
       [1.39996369e-01],
       [1.47716960e-01],
       [1.58913155e-01],
       [1.71945375e-01],
       [1.80186727e-01],
       [1.91521359e-01],
       [2.14169144e-01],
       [2.28417887e-01],
       [2.30316095e-01],
       [2.45037970e-01],
       [2.62180176e-01]])

In [22]:
import pandas as pd

proj_df = pd.DataFrame(eigenvcts[:, zero_eigenvals_index.squeeze()])
proj_df.columns = ['v_' + str(c) for c in proj_df.columns]
proj_df.head()

,v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
0,-0.07785,0.020694,-0.067301,0.018447,-0.005222,0.070510,-0.013832,0.023375,0.056887,0.088584,0.034506,-0.021931,-0.287686,0.086673,0.034768
1,-0.07785,-0.005273,0.147785,-0.321033,-0.051376,-0.004416,-0.006869,0.092596,-0.016712,0.087481,-0.099494,-0.109038,-0.039262,-0.065011,-0.080727
2,-0.07785,0.025084,-0.083119,0.007179,-0.031988,-0.126369,-0.122319,0.050328,0.029667,-0.001549,-0.024688,0.000312,0.016617,-0.007789,0.045015
3,-0.07785,0.025155,-0.061428,0.021548,-0.004252,-0.032361,0.181412,0.156433,-0.069944,-0.024901,-0.000536,-0.001534,0.017898,-0.009375,0.032677
4,-0.07785,0.024212,0.023502,0.045609,0.053455,-0.018485,-0.039076,-0.002526,-0.066741,0.015605,-0.016009,0.017704,0.018869,-0.005075,0.016558


Stack the Eigen Vectors corresponding to the zero Eigen Values in a dataframe *proj_df*. This can be thought of as a $N X K$ matrix where the columns denote an eigen vector and the rows denote a data point.

Apply *K-Means Clustering* with $K = 15$.

In [30]:
from sklearn.cluster import KMeans

def run_k_means(df, n_clusters):
    k_means = KMeans(random_state=25, n_clusters=n_clusters)
    k_means.fit(df)
    cluster = k_means.predict(df)
    return cluster

cluster = run_k_means(proj_df, n_clusters=K) +1

*run_k_means* applies `K-Means Clustering` on *proj_df* with number of clusters = $3$.The clustering of the data points is returned in the variable *'cluster'*.

Display clusters formed

In [31]:
print(cluster)

[ 9  1 14 13  7  8 15  7  7  5  3 12 13  7  8 11  7  5 10  2  5  8  2 10
  4 11  7 14 10  4 12  1  4 13 14  7  7 15  7  7  6  7  2  7  1  4  4  4
  4  1  7 14  2  7  1  3  2  4  3  7  4  2  4 13 10  3 14  7  7 12  3  7
 11  7 10  5  9 14  7  7 13  5  7  3 15  7  4  7  9  7  7 11  7  4  7 12
 11  1  9  2  2 13  4  2  2 10  7  5 15  4  4 12  8  7  7  7  7  3  7 14
 11  2  3  7  1 11  2  7  1  5 15  7  2  7  5 10  3  4 12  4 12  6 11  2
  8  3  5  7  8  7 15  8  8 14 10 13  1 14  3 10  9  7 13  3  5]


As we can see, the data points have been clustered into $fifteen$ subspaces: $1$ to $15$ corresponding to the $fifteen$ subspaces that we have considered.

In [33]:
pred = np.asarray(cluster)

### Calculate Performance

In [34]:
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import normalized_mutual_info_score
print("ARI = " + str(adjusted_rand_score(orig_label, pred)))
print("NMI = " + str(normalized_mutual_info_score(orig_label, pred)))

ARI = 0.3605097661590404
NMI = 0.6786820517912824


In the above code block, we calculate the `Adjusted random score` and the `normalized mutual info score` between the `original` and the `predicted` labels for the various data points.